<a href="https://colab.research.google.com/github/VinnyStrokes/gold-mining-screener/blob/main/gold_mining_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install yfinance --quiet

import pandas as pd
import yfinance as yf
import time

# Seed list of gold/silver mining tickers from GDX, GDXJ, and other ETFs
tickers = [
    "NEM", "AEM", "GOLD", "WPM", "FNV", "AGI", "PAAS", "KGC", "SSRM", "BTG",
    "OR", "AU", "NGD", "EGO", "IAG", "HL", "MGG.TO", "LYD.TO", "SILV", "SBSW"
]

# Function to fetch fundamentals
def fetch_fundamentals(ticker):
    try:
        info = yf.Ticker(ticker).info
        market_cap = info.get('marketCap')
        pe = info.get('trailingPE')
        industry = info.get('industry')
        name = info.get('shortName') or info.get('longName')
        return {
            'Ticker': ticker,
            'Name': name,
            'MarketCap': market_cap,
            'PE': pe,
            'Industry': industry
        }
    except Exception as e:
        print(f"⚠️ Error fetching {ticker}: {e}")
        return None

# Fetch and collect data
fundamentals = []
for i, t in enumerate(tickers):
    if i % 10 == 0 and i > 0:
        print(f"⏳ Processed {i} tickers...")
        time.sleep(2)
    data = fetch_fundamentals(t)
    if data and data['MarketCap'] and data['PE'] and data['Industry']:
        fundamentals.append(data)

df_fund = pd.DataFrame(fundamentals)
print(f"\n✅ Fetched fundamentals for {len(df_fund)} tickers\n")
print(df_fund.head())

# Screen for undervalued players in each industry
results = []
industry_groups = df_fund.groupby('Industry')

for industry, group in industry_groups:
    max_mcap = group['MarketCap'].max()
    largest_pe = group.loc[group['MarketCap'].idxmax(), 'PE']
    filtered = group[(group['MarketCap'] < max_mcap) & (group['PE'] < largest_pe)]
    if not filtered.empty:
        filtered = filtered.copy()
        filtered['Largest_PE'] = largest_pe
        filtered['PE_Spread'] = largest_pe - filtered['PE']
        filtered['Largest_MCap'] = max_mcap
        results.append(filtered)

# Display results
if results:
    hidden_gems = pd.concat(results).sort_values(by='PE_Spread', ascending=False)
    display_cols = ['Ticker', 'Name', 'Industry', 'MarketCap', 'PE', 'Largest_MCap', 'Largest_PE', 'PE_Spread']
    print(f"\n🎯 Hidden gems based on P/E vs market cap leader:\n")
    print(hidden_gems[display_cols].head(30))
else:
    print("😕 No hidden gems found with current criteria.")


⏳ Processed 10 tickers...
⚠️ Error fetching MGG.TO: HTTP Error 404: 
⚠️ Error fetching SILV: HTTP Error 404: 

✅ Fetched fundamentals for 15 tickers

  Ticker                          Name    MarketCap         PE Industry
0    NEM           Newmont Corporation  58822045696  12.038724     Gold
1    AEM    Agnico Eagle Mines Limited  59345776640  25.078726     Gold
2   GOLD      Barrick Gold Corporation  32429015040  14.287879     Gold
3    WPM  Wheaton Precious Metals Corp  39107047424  62.897808     Gold
4    FNV     Franco-Nevada Corporation  32590428160  52.329193     Gold

🎯 Hidden gems based on P/E vs market cap leader:

   Ticker                       Name Industry    MarketCap         PE  \
13    IAG        Iamgold Corporation     Gold   4077125632   4.684931   
0     NEM        Newmont Corporation     Gold  58822045696  12.038724   
12    EGO  Eldorado Gold Corporation     Gold   4127892224  12.102410   
2    GOLD   Barrick Gold Corporation     Gold  32429015040  14.287879   
10

In [ ]:
!pip install yfinance
